In [1]:
from google.colab import files
import json

# Upload your analysis prompt file
print("📁 Upload your analysis_prompt_20251120_002408.txt file:")
uploaded = files.upload()

# Read the prompt
prompt_file = list(uploaded.keys())[0]
with open(prompt_file, 'r', encoding='utf-8') as f:
    analysis_prompt = f.read()

print(f"✅ Loaded prompt: {len(analysis_prompt)} characters")

📁 Upload your analysis_prompt_20251120_002408.txt file:


Saving analysis_prompt_20251120_002408.txt to analysis_prompt_20251120_002408.txt
✅ Loaded prompt: 547991 characters


In [2]:
!pip install -q anthropic google-generativeai groq together huggingface_hub requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.5 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata

# Store your API keys in Colab Secrets (left sidebar → 🔑 Secrets)
# Or enter them directly here (less secure):
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')        # Gemini
GROQ_API_KEY = userdata.get('GROQ_API_KEY')            # Groq (Free)
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY') # OpenRouter
HUGGINGFACE_API_KEY = userdata.get('HUGGINGFACE_API_KEY') # HuggingFace

print("🔑 API Keys loaded!")

🔑 API Keys loaded!


In [4]:
!pip install -q anthropic google-generativeai groq together huggingface_hub requests

In [ ]:
import google.generativeai as genai
from groq import Groq
from huggingface_hub import InferenceClient
import requests
import json
from datetime import datetime
import time
from google.colab import userdata, files

# Configure APIs
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
HUGGINGFACE_API_KEY = userdata.get('HUGGINGFACE_API_KEY')

# Initialize clients
genai.configure(api_key=GOOGLE_API_KEY)
groq_client = Groq(api_key=GROQ_API_KEY)
hf_client = InferenceClient(token=HUGGINGFACE_API_KEY)

results = {}
errors = {}

print("🏥 MULTI-LLM HOSPITAL ANALYSIS STARTING...\n")
print("="*60)

# Function to create a smaller, focused prompt
def create_focused_prompt(full_data):
    """Create a smaller prompt by summarizing data"""

    # Extract just the key information
    hospitals = list(full_data['data'].keys())

    # Build a condensed version with sample reviews only
    condensed_data = {}
    for hospital, data in full_data['data'].items():
        condensed_data[hospital] = {
            'review_count': data['review_count'],
            'news_count': data['news_count'],
            'sample_reviews': data['reviews'],
            'sample_news': data['news']
        }

    prompt = f"""I have collected review and news data for {len(hospitals)} hospitals in Delhi:
{', '.join(hospitals)}

DATA SUMMARY:
{chr(10).join([f"{h}: {condensed_data[h]['review_count']} reviews, {condensed_data[h]['news_count']} news" for h in hospitals])}

SAMPLE DATA:
{json.dumps(condensed_data, indent=2)}

Based on this sample data, provide:

A. OVERALL RANKING (1-{len(hospitals)}):
- Rank hospitals best to worst
- Give each a score out of 10
- Justify with specific review quotes in [brackets]

B. TOP 3 STRENGTHS & WEAKNESSES for each hospital:
- Use actual review quotes as evidence

C. BEST HOSPITAL FOR:
- Emergency care
- Planned surgery
- Budget patients
- Cancer treatment
- Heart disease
- Orthopedics

D. KEY RECOMMENDATIONS:
- Which hospitals to prefer for specific needs
- Which to avoid and why

Be specific and quote actual reviews in [brackets]."""

    return prompt

# Helper function for OpenRouter API calls
def call_openrouter(model_id, model_name, prompt, max_tokens=4000, timeout=120):
    """Generic function to call OpenRouter API"""
    try:
        print(f"\n🤖 Running: {model_name}...")
        start = time.time()

        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {OPENROUTER_API_KEY}",
                "Content-Type": "application/json",
                "HTTP-Referer": "https://github.com/yourusername",
                "X-Title": "Hospital Analysis"
            },
            json={
                "model": model_id,
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": max_tokens
            },
            timeout=timeout
        )

        response.raise_for_status()
        response_data = response.json()

        elapsed_time = time.time() - start
        result = {
            'response': response_data['choices'][0]['message']['content'],
            'time': elapsed_time,
            'model': model_name
        }
        print(f"✅ {model_name} completed in {elapsed_time:.2f}s")
        return result

    except Exception as e:
        print(f"❌ {model_name} failed: {e}")
        raise e

# Load your full data
print("📁 Upload your combined hospital data JSON file:")
uploaded = files.upload()
data_file = list(uploaded.keys())[0]

with open(data_file, 'r', encoding='utf-8') as f:
    full_hospital_data = json.load(f)

# Create focused prompt
analysis_prompt = create_focused_prompt(full_hospital_data)
print(f"\n✅ Created focused prompt: ~{len(analysis_prompt)} characters")
print(f"Estimated tokens: ~{len(analysis_prompt.split())}")

# 1. GEMINI 1.5 FLASH
try:
    print("\n🤖 Running: Gemini 1.5 Flash...")
    start = time.time()

    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(analysis_prompt)

    results['gemini_flash'] = {
        'response': response.text,
        'time': time.time() - start,
        'model': 'Gemini 1.5 Flash (Google)'
    }
    print(f"✅ Gemini Flash completed in {results['gemini_flash']['time']:.2f}s")

except Exception as e:
    errors['gemini_flash'] = str(e)
    print(f"❌ Gemini Flash failed: {e}")

# 2. GROQ - Llama 3.1 70B Versatile
try:
    print("\n🤖 Running: Groq Llama 3.1 70B...")
    start = time.time()

    response = groq_client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[{"role": "user", "content": analysis_prompt}],
        temperature=0.7,
        max_tokens=4000
    )

    results['groq_llama_70b'] = {
        'response': response.choices[0].message.content,
        'time': time.time() - start,
        'model': 'Llama 3.1 70B (Groq)'
    }
    print(f"✅ Groq Llama 70B completed in {results['groq_llama_70b']['time']:.2f}s")

except Exception as e:
    errors['groq_llama_70b'] = str(e)
    print(f"❌ Groq Llama 70B failed: {e}")

# 3. GROQ - Llama 3.2 90B
try:
    print("\n🤖 Running: Groq Llama 3.2 90B...")
    start = time.time()

    response = groq_client.chat.completions.create(
        model="llama-3.2-90b-text-preview",
        messages=[{"role": "user", "content": analysis_prompt}],
        temperature=0.7,
        max_tokens=4000
    )

    results['groq_llama_90b'] = {
        'response': response.choices[0].message.content,
        'time': time.time() - start,
        'model': 'Llama 3.2 90B (Groq)'
    }
    print(f"✅ Groq Llama 90B completed in {results['groq_llama_90b']['time']:.2f}s")

except Exception as e:
    errors['groq_llama_90b'] = str(e)
    print(f"❌ Groq Llama 90B failed: {e}")

# OPENROUTER MODELS (EXPANDED SECTION)
print("\n" + "="*60)
print("🌐 OPENROUTER MODELS")
print("="*60)

# OpenRouter Model Definitions
openrouter_models = [
    # Anthropic Models
    ("anthropic/claude-3.5-sonnet", "Claude 3.5 Sonnet"),
    ("anthropic/claude-3.5-haiku", "Claude 3.5 Haiku"),
    ("anthropic/claude-3-opus", "Claude 3 Opus"),
    ("anthropic/claude-3-sonnet", "Claude 3 Sonnet"),

    # Meta Llama Models
    ("meta-llama/llama-3.1-405b-instruct", "Llama 3.1 405B"),
    ("meta-llama/llama-3.1-70b-instruct", "Llama 3.1 70B"),
    ("meta-llama/llama-3.3-70b-instruct", "Llama 3.3 70B"),

    # Google Models
    ("google/gemini-pro-1.5", "Gemini Pro 1.5"),
    ("google/gemini-flash-1.5", "Gemini Flash 1.5"),

    # OpenAI Models
    ("openai/gpt-4-turbo", "GPT-4 Turbo"),
    ("openai/gpt-4o", "GPT-4o"),
    ("openai/gpt-4o-mini", "GPT-4o Mini"),

    # Mistral Models
    ("mistralai/mistral-large", "Mistral Large"),
    ("mistralai/mistral-medium", "Mistral Medium"),

    # Other Notable Models
    ("qwen/qwen-2.5-72b-instruct", "Qwen 2.5 72B"),
    ("deepseek/deepseek-chat", "DeepSeek Chat"),
    ("cohere/command-r-plus", "Cohere Command R+"),
    ("perplexity/llama-3.1-sonar-large-128k-online", "Perplexity Sonar Large"),
]

# Run each OpenRouter model
for model_id, model_name in openrouter_models:
    key = f"openrouter_{model_name.lower().replace(' ', '_').replace('.', '_').replace('-', '_').replace('+', 'plus')}"
    try:
        results[key] = call_openrouter(
            model_id=model_id,
            model_name=f"{model_name} (OpenRouter)",
            prompt=analysis_prompt
        )
    except Exception as e:
        errors[key] = str(e)

# 4. HUGGINGFACE - Qwen 2.5 72B
try:
    print("\n🤖 Running: HuggingFace Qwen 2.5 72B...")
    start = time.time()

    response = hf_client.chat_completion(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=[{"role": "user", "content": analysis_prompt}],
        max_tokens=4000
    )

    results['hf_qwen'] = {
        'response': response.choices[0].message.content,
        'time': time.time() - start,
        'model': 'Qwen 2.5 72B (HuggingFace)'
    }
    print(f"✅ HuggingFace Qwen completed in {results['hf_qwen']['time']:.2f}s")

except Exception as e:
    errors['hf_qwen'] = str(e)
    print(f"❌ HuggingFace Qwen failed: {e}")

# 5. HUGGINGFACE - Meta Llama 3.1 70B
try:
    print("\n🤖 Running: HuggingFace Llama 3.1 70B...")
    start = time.time()

    response = hf_client.chat_completion(
        model="meta-llama/Llama-3.1-70B-Instruct",
        messages=[{"role": "user", "content": analysis_prompt}],
        max_tokens=4000
    )

    results['hf_llama'] = {
        'response': response.choices[0].message.content,
        'time': time.time() - start,
        'model': 'Llama 3.1 70B (HuggingFace)'
    }
    print(f"✅ HuggingFace Llama completed in {results['hf_llama']['time']:.2f}s")

except Exception as e:
    errors['hf_llama'] = str(e)
    print(f"❌ HuggingFace Llama failed: {e}")

print("\n" + "="*60)
print("📊 ANALYSIS COMPLETE!")
print(f"✅ Successful: {len(results)} models")
print(f"❌ Failed: {len(errors)} models")

if errors:
    print("\n⚠️ Failed models:")
    for model, error in errors.items():
        print(f"  - {model}: {error[:100]}...")

# Save results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_data = {
    'timestamp': timestamp,
    'successful_models': len(results),
    'failed_models': len(errors),
    'results': results,
    'errors': errors,
    'prompt_used': analysis_prompt
}

output_file = f'hospital_analysis_{timestamp}.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\n💾 Results saved to: {output_file}")
files.download(output_file)

# Save individual markdown reports
for model_name, data in results.items():
    filename = f"{model_name}_analysis.md"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"# Hospital Analysis - {data['model']}\n\n")
        f.write(f"**Generated:** {timestamp}\n")
        f.write(f"**Processing Time:** {data['time']:.2f}s\n\n")
        f.write("---\n\n")
        f.write(data['response'])
    print(f"📄 Saved: {filename}")
    files.download(filename)

# Print summary statistics
print("\n" + "="*60)
print("📈 MODEL PERFORMANCE SUMMARY")
print("="*60)
if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1]['time'])
    print("\n⚡ Fastest to Slowest:")
    for i, (model_name, data) in enumerate(sorted_results, 1):
        print(f"{i}. {data['model']}: {data['time']:.2f}s")

# Print preview
print("\n" + "="*60)
print("RESULTS PREVIEW")
print("="*60)
for model_name, data in results.items():
    print(f"\n🤖 {data['model']}")
    print(f"⏱️  {data['time']:.2f}s")
    print("-"*60)
    print(data['response'][:400] + "...\n")

🏥 MULTI-LLM HOSPITAL ANALYSIS STARTING...

📁 Upload your combined hospital data JSON file:


Saving all_hospitals_combined_20251120_002403.json to all_hospitals_combined_20251120_002403.json

✅ Created focused prompt: ~64140 characters
Estimated tokens: ~9591

🤖 Running: Gemini 1.5 Flash...


❌ Gemini Flash failed: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

🤖 Running: Groq Llama 3.1 70B...
❌ Groq Llama 70B failed: Error code: 400 - {'error': {'message': 'The model `llama-3.1-70b-versatile` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

🤖 Running: Groq Llama 3.2 90B...
❌ Groq Llama 90B failed: Error code: 400 - {'error': {'message': 'The model `llama-3.2-90b-text-preview` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use i

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Model meta-llama/Llama-3.1-70B-Instruct is in staging mode for provider hyperbolic. Meant for test purposes only.


❌ HuggingFace Qwen failed: 504 Server Error: Gateway Time-out for url: https://router.huggingface.co/novita/v3/openai/chat/completions

🤖 Running: HuggingFace Llama 3.1 70B...
✅ HuggingFace Llama completed in 11.54s

📊 ANALYSIS COMPLETE!
✅ Successful: 7 models
❌ Failed: 16 models

⚠️ Failed models:
  - gemini_flash: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%2...
  - groq_llama_70b: Error code: 400 - {'error': {'message': 'The model `llama-3.1-70b-versatile` has been decommissioned...
  - groq_llama_90b: Error code: 400 - {'error': {'message': 'The model `llama-3.2-90b-text-preview` has been decommissio...
  - openrouter_claude_3_5_sonnet: 402 Client Error: Payment Required for url: https://openrouter.ai/api/v1/chat/completions...
  - openrouter_claude_3_opus: 402 Client Error: Payment Required for url: https://openrouter.ai/api/v1/chat/completions...
  - openrouter_claude_3_sonnet: 404 Client Error: Not Found for url: https://ope

...

<IPython.core.display.Javascript object>

📄 Saved: hf_llama_analysis.md


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📈 MODEL PERFORMANCE SUMMARY

⚡ Fastest to Slowest:
1. Llama 3.1 70B (HuggingFace): 11.54s
2. Llama 3.1 70B (OpenRouter): 16.97s
3. Claude 3.5 Haiku (OpenRouter): 18.18s
4. GPT-4o Mini (OpenRouter): 39.19s
5. Llama 3.3 70B (OpenRouter): 46.88s
6. DeepSeek Chat (OpenRouter): 52.02s
7. Qwen 2.5 72B (OpenRouter): 91.20s

RESULTS PREVIEW

🤖 Claude 3.5 Haiku (OpenRouter)
⏱️  18.18s
------------------------------------------------------------
A. OVERALL RANKING (Out of 10):

1. Apollo (7.5/10)
- Mostly positive reviews about patient care
- [Good for international patients with skilled doctors]
- [Excellent treatment and compassionate staff]

2. Max (6/10)
- Mixed reviews with more negative experiences
- [Some positive staff interactions]
- [Significant complaints about billing and negligence]

3. AIIMS (5.5/10)
- Generally positive ins...


🤖 Llama 3.1 70B (OpenRouter)
⏱️  16.97s
------------------------------------------------------------
A. OVERALL RANKING (1-7):

1. Max Hospital (Score: 6